In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
keyword_df = pd.read_pickle('../data/keywords.pk')
words_df = pd.read_pickle('../data/recwords.pk')

In [31]:
def lower_case_exc(something):
    if type(something)==str:
        return something.lower()
    else:
        return something

In [9]:
recipe_ids = list(set(keyword_df.index).intersection(set(words_df.index)))
keyword_df = keyword_df[keyword_df.index.isin(recipe_ids)]
words_df = words_df[words_df.index.isin(recipe_ids)]

In [32]:
for col in words_df.columns:
    words_df[col]=words_df[col].apply(lambda x:lower_case_exc(x))

In [54]:
words_df['Keywords_orig']=[keyword_df['Keywords'][recipe] for recipe in words_df.index]

In [35]:
categories = list({w for w in words_df.RecipeCategory if type(w)==str})

In [37]:
len(words_df.RecipeCategory.unique())

272

In [17]:
keyword_list_orig = []
for kwl in keyword_df.Keywords:
    keyword_list_orig+=list(kwl)

keyword_list_orig_nr = list(set(keyword_list_orig))

In [45]:
words_df.head()

,Name,Description,RecipeCategory
RecipeId,,,
38,low-fat berry blue frozen dessert,make and share this low-fat berry blue frozen ...,frozen desserts
40,best lemonade,this is from one of my first good house keepi...,beverages
42,cabbage soup,make and share this cabbage soup recipe from f...,vegetable
44,warm chicken a la king,i copied this one out of a friend's book so ma...,chicken
45,buttermilk pie with gingersnap crumb crust,make and share this buttermilk pie with ginger...,pie


In [38]:
extrakeywords=['pasta','chicken','rice','cheap','simple','burger','veggie','fried','sauce','soup','chocolate','pudding','taco','hummus','healthy',
               'cake','egg','italian','mushroom','casserole','french toast','pancake','waffle','salad','pie','noodle','ramen','pizza']

In [40]:
keyword_list_new = list(set(keyword_list_orig_nr+extrakeywords+categories))

In [50]:
keywords_in_titles = []
for name in words_df.Name:
    keywords_in_titles.append([word for word in keyword_list_new if word in name])
words_df['KeywordsTitle']=keywords_in_titles

In [52]:
keywords_in_desc = []
for desc in words_df.Description:
    keywords_in_desc.append([word for word in keyword_list_new if word in desc])
words_df['KeywordsDesc']=keywords_in_desc

In [57]:
new_keyword_data = []
for rec in words_df.index:
    kws_new = []
    if type(words_df['RecipeCategory'][rec])==str:
        kws_new.append(words_df['RecipeCategory'][rec])
    for col in ['KeywordsTitle','KeywordsDesc','Keywords_orig']:
        kws_new+=list(words_df[col][rec])
    kws_new = list(set(kws_new))
    new_keyword_data.append(kws_new)
words_df['Keywords'] = new_keyword_data

In [61]:
words_df['Keywords_orig'].apply(len).mean()

5.70336575332915

In [64]:
words_df[['Name','KeywordsTitle','Keywords']].to_pickle('../data/keywords.pk')

{'easy', 'meat'}